# BASE DE DATOS RNA: más de 40 tablas

In [1]:
import sys
sys.path.append('C:/Users/plane/OneDrive/Escritorio/COMPUTING SCIENCE/Apuntes_plantillas/openai_API')
from calcular_tokens import num_tokens_from_messages
from utils import conect_to_bbdd, store_simple_metadata_in_json_str, store_full_metadata_in_json_str


## Prueba básica de conexion a la base de datos pública

In [8]:
def conect_to_bbdd__(host, database, user, password, port= '5432'):
    
    from sqlalchemy import create_engine
    from sqlalchemy.orm import sessionmaker
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
    Session = sessionmaker(bind= engine)

    return Session()


try:
    from sqlalchemy import text
    session = conect_to_bbdd__(host= 'hh-pgsql-public.ebi.ac.uk',database="pfmegrnargs", user="reader", password="NWDMCE5xdipIjRrp", port= '5432', )

    consulta = text('SELECT * FROM rna LIMIT 10;')
    registros = session.execute(consulta).fetchall()
    
    # print(type(registros))
    # texto_registros = "\n".join([str(registro) for registro in registros])
    
    for registro in registros: 
        print(registro)

except Exception as e:
    print(f"Error al conectar: {e}")
finally:
    session.close()

(25439783, 'URS0001842E27', datetime.datetime(2019, 12, 2, 13, 26, 59, 288824), 'rnacen', '5723224CF6E25978', 559, 'AAAACCATTAGGGAATCTTCGGCAATGGACGAAAGTCTGACCGAGCAACGCCGCGTGAGTGATGAAGGTTTTCGGATCGTAAAACTCTGTTGTTAGAGAAGAACAAGTACCGTTTGAATAAGGCGGTACCTTGACGGTACCTAACGAGA ... (261 characters truncated) ... AGGCGCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGAGTGCTAGGTGTTAGGGGTTTCGATGCCCTTAGTGCCGACGTTAACACATTAAGCACTCCGCCTGGGGAGTACGGCCGCAAGGCTG', None, 'e65e80788e833810f64a9173dc1069f0')
(25439784, 'URS0001842E28', datetime.datetime(2019, 12, 2, 13, 26, 59, 288885), 'rnacen', 'F68D4F1CB1187070', 177, 'GAGGCAGCAGTAAGGAATATTGGGCAATGGGCGCAAGCCTGACCCAGCCATGCCGCGTGAAGGATGACGGCCCTATGGGTTGTAAACTTCCTTTGTACGGGAAAAAACCACCGGTCGTGAACCGGTGCTGATGGTACCGTAAGAATAAGCATCGGCTAACTCCGTGCCAGCAGCCGC', None, 'e65e82d5cc0239f5cd302d0e56ca02d7')
(25439785, 'URS0001842E29', datetime.datetime(2019, 12, 2, 13, 26, 59, 288920), 'rnacen', '55B5A1FC7AC2C462', 423, 'CTACGGGGGGCTCCAGTCGGGAAACTTTTGCAATGCGCGAAAGCGTGACAA

## Comprobando la cantidad de tokens de la BBDD 

In [10]:

try: 
    engine = conect_to_bbdd(host= 'hh-pgsql-public.ebi.ac.uk', bbdd_name="pfmegrnargs", user="reader", password="NWDMCE5xdipIjRrp", port= '5432',)
    full_metadata_json_str= store_full_metadata_in_json_str(engine)
    simple_metadata= store_simple_metadata_in_json_str(engine)

    simple_md_prompt = [
        {'role': 'system', 'content': f'Eres un asistente experto en bases de datos, que convierte peticiones de lenguaje natural a código SQL.\
                                       AQUI TIENES LO METADATOS DE LA BASE DE DATOS: \n{simple_metadata}'},      
        {'role': 'user',   'content': 'una consulta en lenguaje natural qualquiera ojito parao'},
        {'role': 'assistant', 'content': 'Devuélveme SOLO EL CODIGO SQL y en este formato: \n```sql\nEL CODIGO SQL;\n```'}
    ]

    full_md_prompt = [
            {'role': 'system', 'content': f'Eres un asistente experto en bases de datos, que convierte peticiones de lenguaje natural a código SQL.\
                                        AQUI TIENES LO METADATOS DE LA BASE DE DATOS: \n{full_metadata_json_str}'},      
            {'role': 'user',   'content': 'una consulta en lenguaje natural qualquiera ojito parao'},
            {'role': 'assistant', 'content': 'Devuélveme SOLO EL CODIGO SQL y en este formato: \n```sql\nEL CODIGO SQL;\n```'}
    ]

    tokens_en_simple_md_prompt = num_tokens_from_messages(simple_md_prompt)
    tokens_en_full_md_prompt = num_tokens_from_messages(full_md_prompt)
    
    

    #print(simple_metadata)
    #print()
    print(f'con los metadatos simples hay un total de: {tokens_en_simple_md_prompt} tokens')
    
    #print(full_metadata_json_str)
    print(f'con los metadatas completos hay un total de: {tokens_en_full_md_prompt} tokens')
    
    

    
    # metadata = MetaData()
    # metadata.reflect(engine)

    # simple_metadata = {table.name: [column.name for column in table.columns] for table in metadata.tables.values()}

    # full_metadata = {}
    # for table in metadata.tables.values():
    #         # Por cada tabla, se almacena información detallada de sus columnas en el diccionario 'tablas_columnas'.
    #         # Se crea una clave en el diccionario para cada nombre de tabla.
    #     full_metadata[table.name] = {                                     
    #     "columnas": {column.name: {                                       # dentro de cada tabla se crea un diccionario de 
    #                         "tipo": str(column.type),                     # Tipo de dato de la columna.
    #                         "nulo": column.nullable,                      # Booleano que indica si la columna acepta valores nulos.
    #                         "clave_primaria": column.primary_key,         # Booleano que indica si la columna es una clave primaria
    #                         "clave_foranea": bool(column.foreign_keys)}   # Booleano que indica si la columna es una clave foránea.
    #                 for column in table.columns}                          # Este bucle interno itera a través de todas las columnas de la tabla.
    #     }
        
    # consulta_sql = "SELECT * FROM actor"          # crear la consulta
    # df = pd.read_sql(consulta_sql, engine)         # read_sql para pasar la consulta a dataframe
    

except Exception as e: 
    print(f"Ocurrio un error al establecer la conexion {e}")
    
finally:
    engine.dispose()

con los metadatos simples hay un total de: 18122 tokens
con los metadatas completos hay un total de: 131109 tokens


In [11]:
print(simple_metadata)

{
    "ensembl_assembly": [
        "assembly_id",
        "assembly_full_name",
        "gca_accession",
        "assembly_ucsc",
        "common_name",
        "taxid",
        "ensembl_url",
        "division",
        "blat_mapping",
        "example_chromosome",
        "example_end",
        "example_start",
        "subdomain",
        "selected_genome"
    ],
    "go_term_annotations": [
        "go_term_annotation_id",
        "rna_id",
        "qualifier",
        "ontology_term_id",
        "evidence_code",
        "assigned_by",
        "extensions"
    ],
    "load_karyotypes": [
        "assembly_id",
        "karyotype"
    ],
    "load_rnc_accessions": [
        "accession",
        "parent_ac",
        "seq_version",
        "feature_start",
        "feature_end",
        "feature_name",
        "ordinal",
        "division",
        "keywords",
        "description",
        "species",
        "organelle",
        "classification",
        "project",
        "is_compo